In [2]:
import cv2
import os
import numpy as np
import pandas as pd

In [3]:
path_dir = "C:\\Users\\caiosmedeiros\\Downloads\\train"
files = os.listdir(path_dir)
files

['2012-09-11_15_16_58_jpg.rf.61d961a86c9a16694403dfcb72cd450c.jpg',
 '2012-09-11_15_27_08_jpg.rf.0f38b15658ce17d10ce40a992adae5ba.jpg',
 '2012-09-11_15_29_29_jpg.rf.3819d0bdff07fb09e87e5cc7d51dc662.jpg',
 '2012-09-11_15_31_50_jpg.rf.b33fc3bf053074e8dab4fa16108b950c.jpg',
 '2012-09-11_15_38_53_jpg.rf.bcdabdb175b85ebb981248ddc666e1d7.jpg',
 '2012-09-11_15_43_35_jpg.rf.6a3a3c38c0b7e56174dff7a2028b5ccd.jpg',
 '2012-09-11_15_55_21_jpg.rf.b96c1ba21a88e75e460e8149a886aa30.jpg',
 '2012-09-11_16_02_25_jpg.rf.eba5df67eba60842066f6da6b7e3f916.jpg',
 '2012-09-11_16_10_41_jpg.rf.acfbc89930f9e634223cc4bc9b0ebb06.jpg',
 '2012-09-11_16_15_24_jpg.rf.977a78d181720a089f4ef4abcf7926a3.jpg',
 '2012-09-11_16_20_06_jpg.rf.901497d9edde7c48c8020334fb8d9aae.jpg',
 '2012-09-11_16_24_53_jpg.rf.790f7cbe2158bf835d4b85f757d9d6d5.jpg',
 '2012-09-11_16_29_30_jpg.rf.d2f66cfac7ebb008dec63e9f87302fc4.jpg',
 '2012-09-11_16_34_18_jpg.rf.c7f0037f799f9fc4e0d32081e1f12ba4.jpg',
 '2012-09-11_16_38_55_jpg.rf.5c9b1a58fdc3bf2df7f

In [4]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [5]:
file_path = os.path.join(path_dir, 'C:\\Users\\caiosmedeiros\\Downloads\\test.jpg')
image = cv2.imread(file_path)

detections = model.predict(image)

centers = []
boxes = []

for detection in detections:
    for box in detection.boxes:
        class_id = int(box.cls[0])
        class_name = detection.names[class_id]
        if class_name == 'car':
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w = x2-x1
            h=y2-y1
            center = (int(x1+(w/2)), int(y1+(h/2)))
            centers.append(center)
            boxes.append([(x1,y1), (x2,y2)])
            cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0))
cv2.imshow("Imagem", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 384x640 43 cars, 1 bus, 124.1ms
Speed: 14.0ms preprocess, 124.1ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)


In [6]:
import cv2.typing


def draw(image:cv2.typing.MatLike, idx:int):

    pt1, pt2 = map(tuple, boxes[idx])
    center = centers[idx]
    cv2.rectangle(image, pt1, pt2, (0, 255, 0))
    cv2.ellipse(image, center, axes=(3, 3), angle=0, startAngle=0, endAngle=360, color=(0, 0, 255), thickness=2)

In [14]:
import numpy as np
from scipy.spatial.distance import cosine, euclidean
from sklearn.preprocessing import minmax_scale


# --- Vetores de direção e centros ---
directions = []
centers = []
for (x1, y1), (x2, y2) in boxes:
    top_center = np.array([(x1 + x2) / 2, y1])
    bottom_center = np.array([(x1 + x2) / 2, y2])
    vector = bottom_center - top_center
    directions.append(vector)
    center = (top_center + bottom_center) / 2
    centers.append(center)

# Índice de referência
idx_ref = 10
ref_vector = directions[idx_ref]
ref_center = centers[idx_ref]

# --- Calcula distância cosseno e euclidiana ---
cosine_distances = []
euclidean_distances = []

for i in range(len(directions)):
    if i == idx_ref:
        cosine_distances.append(float('inf'))
        euclidean_distances.append(float('inf'))
    else:
        cos_dist = cosine(ref_vector, directions[i])
        euc_dist = euclidean(ref_center, centers[i])
        cosine_distances.append(cos_dist)
        euclidean_distances.append(euc_dist)

# Normaliza a distância euclidiana apenas para os valores finitos
euclidean_distances_np = np.array(euclidean_distances)
mask = np.isfinite(euclidean_distances_np)
euclidean_distances_norm = euclidean_distances_np.copy()
euclidean_distances_norm[mask] = minmax_scale(euclidean_distances_np[mask])
# O valor no índice de referência permanece infinito

# --- Score combinado ---
alpha = 0.1
beta = 0.5
scores = [alpha * cos + beta * euc for cos, euc in zip(cosine_distances, euclidean_distances_norm)]

# Índice do carro mais alinhado e próximo (o de menor score)
idx_alinhado_proximo = int(np.argmin(scores))

print(f"Carro {idx_ref} está mais alinhado e próximo com o carro {idx_alinhado_proximo}")

# --- Função para desenhar as bounding boxes com os índices ---
def draw_boxes(image, boxes, idx_ref, idx_alinhado_proximo):
    for idx, ((x1, y1), (x2, y2)) in enumerate(boxes):
        # Define a cor: verde para o carro de referência, vermelho para o carro com menor score e azul para os demais.
        if idx == idx_ref:
            color = (0, 255, 0)  # Verde
        elif idx == idx_alinhado_proximo:
            color = (0, 0, 255)  # Vermelho
        else:
            color = (255, 0, 0)  # Azul
        
        # Desenha o retângulo da bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        # Escreve o índice próximo à bounding box
        cv2.putText(image, f"{idx}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# --- Carrega a imagem e desenha as bounding boxes ---
file_path = os.path.join(path_dir, 'C:\\Users\\caiosmedeiros\\Downloads\\test.jpg')  # Substitua pelo caminho correto da sua imagem
image = cv2.imread(file_path)
if image is None:
    print("Erro ao carregar a imagem. Verifique o caminho.")
else:
    draw_boxes(image, boxes, idx_ref, idx_alinhado_proximo)
    cv2.imshow("Bounding Boxes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Carro 10 está mais alinhado e próximo com o carro 4
